In [2]:
# import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset from the excel file
file_path = 'all-euro-data-2023-2024.xlsx'
df_epl = pd.read_excel(file_path, sheet_name='E0')
df_bundesliga = pd.read_excel(file_path, sheet_name='D1')
df_laliga = pd.read_excel(file_path, sheet_name='SP1')
df_ligue1 = pd.read_excel(file_path, sheet_name='F1')
df_seriea = pd.read_excel(file_path, sheet_name='I1')


# concatenate all the dataframes into one
df = pd.concat([df_epl, df_bundesliga, df_laliga, df_ligue1, df_seriea], ignore_index=True)

df.tail(), df.shape
                   

(    Div       Date      Time    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  \
 771  I1 2023-12-17  14:00:00  Fiorentina       Verona     1     0   H     0   
 772  I1 2023-12-17  14:00:00     Udinese     Sassuolo     2     2   D     1   
 773  I1 2023-12-17  17:00:00     Bologna         Roma     2     0   H     1   
 774  I1 2023-12-17  19:45:00       Lazio        Inter     0     2   A     0   
 775  I1 2023-12-18  19:45:00    Atalanta  Salernitana     4     1   H     0   
 
      HTAG  ... AvgC<2.5  AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  \
 771     0  ...     1.98 -1.00      1.84      2.09   1.85   2.09     1.85   
 772     0  ...     2.00 -0.25      2.10      1.70   2.20   1.77     2.20   
 773     0  ...     1.44 -0.25      2.13      1.81   2.14   1.81     2.16   
 774     1  ...     1.72  0.50      1.98      1.95   1.99   1.93     2.15   
 775     1  ...     2.39 -1.75      2.02      1.91   2.00   1.90     2.02   
 
      MaxCAHA  AvgCAHH  AvgCAHA  
 771     2.15     1.

## Data Cleaning

In [3]:
# steps:
# 1. Check for missing values
missing_values = df.isnull().sum()

# 2. Check for duplicates
duplicate_rows = df.duplicated().sum()

# Display the results of the checks
missing_values, duplicate_rows

(Div         0
 Date        0
 Time        0
 HomeTeam    0
 AwayTeam    0
            ..
 PCAHA       0
 MaxCAHH     1
 MaxCAHA     1
 AvgCAHH     0
 AvgCAHA     0
 Length: 106, dtype: int64,
 0)

In [4]:
# Dropping columns with a high number of missing values or irrelevant to our analysis
irrelevant_columns = df.columns[df.isnull().sum() > (0.5 * len(df))]  # Columns with more than 50% missing values
df_cleaned = df.drop(columns=irrelevant_columns)

# Convert 'Date' to datetime
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

# Fill missing values in other relevant columns with appropriate values (like median or mode)
# For numerical columns, we use median and for categorical, we use mode
for column in df_cleaned.columns:
    if df_cleaned[column].dtype == 'object':
        df_cleaned[column].fillna(df_cleaned[column].mode()[0], inplace=True)
    else:
        df_cleaned[column].fillna(df_cleaned[column].median(), inplace=True)

# Recheck for missing values
remaining_missing_values = df_cleaned.isnull().sum().sum()

# Basic information after cleaning
remaining_missing_values, df_cleaned.info()

df_cleaned.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Columns: 105 entries, Div to AvgCAHA
dtypes: datetime64[ns](1), float64(82), int64(16), object(6)
memory usage: 636.7+ KB


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2023-08-11,20:00:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,2.01,2.00,1.92,1.95
1,E0,2023-08-12,12:30:00,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,2023-08-12,15:00:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,2023-08-12,15:00:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,2023-08-12,15:00:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88


In [5]:
# Import the LabelEncoder class from the sklearn library
from sklearn.preprocessing import LabelEncoder

# Encode the team names
label_encoder = LabelEncoder()
df_cleaned['HomeTeamEncoded'] = label_encoder.fit_transform(df_cleaned['HomeTeam'])
df_cleaned['AwayTeamEncoded'] = label_encoder.transform(df_cleaned['AwayTeam'])

In [6]:
# function to calculate recent form
def calculate_form_points(team, df_cleaned, num_matches=5):
    # Filter the matches for the team
    team_matches = df_cleaned[(df_cleaned['HomeTeam'] == team) | (df_cleaned['AwayTeam'] == team)]
    # Sort by date to get the most recent matches
    team_matches = team_matches.sort_values(by='Date', ascending=False)
    # Get the last 'num_matches' matches
    recent_matches = team_matches.head(num_matches)
    # Calculate form points
    form_points = 0
    for _, row in recent_matches.iterrows():
        if row['HomeTeam'] == team and row['FTR'] == 'H':
            form_points += 3
        elif row['AwayTeam'] == team and row['FTR'] == 'A':
            form_points += 3
        elif row['FTR'] == 'D':
            form_points += 1
    return form_points

# Apply the function to get recent form for each team in each match
df_cleaned['HomeTeamRecentForm'] = df_cleaned.apply(lambda x: calculate_form_points(x['HomeTeam'], df_cleaned), axis=1)
df_cleaned['AwayTeamRecentForm'] = df_cleaned.apply(lambda x: calculate_form_points(x['AwayTeam'], df_cleaned), axis=1)

# Average Goals per Game
df_cleaned['HomeTeamAvgGoals'] = df_cleaned.groupby('HomeTeam')['FTHG'].transform('mean')
df_cleaned['AwayTeamAvgGoals'] = df_cleaned.groupby('AwayTeam')['FTAG'].transform('mean')


In [7]:
# Points will be calculated similar to form but for the whole season
def calculate_team_points(team, df_cleaned):
    team_matches = df_cleaned[(df_cleaned['HomeTeam'] == team) | (df_cleaned['AwayTeam'] == team)]
    points = 0
    for _, row in team_matches.iterrows():
        if row['HomeTeam'] == team and row['FTR'] == 'H':
            points += 3
        elif row['AwayTeam'] == team and row['FTR'] == 'A':
            points += 3
        elif row['FTR'] == 'D':
            points += 1
    return points

df_cleaned['HomeTeamPoints'] = df_cleaned.apply(lambda x: calculate_team_points(x['HomeTeam'], df_cleaned), axis=1)
df_cleaned['AwayTeamPoints'] = df_cleaned.apply(lambda x: calculate_team_points(x['AwayTeam'], df_cleaned), axis=1)

In [8]:
# Calculate Home Team Win Percentage
home_team_wins = df_cleaned[df_cleaned['FTR'] == 'H'].groupby('HomeTeam').size()
total_home_matches = df_cleaned.groupby('HomeTeam').size()
home_team_win_percentage = home_team_wins / total_home_matches
# 
# Calculate Away Team Win Percentage
away_team_wins = df_cleaned[df_cleaned['FTR'] == 'A'].groupby('AwayTeam').size()
total_away_matches = df_cleaned.groupby('AwayTeam').size()
away_team_win_percentage = away_team_wins / total_away_matches
# 
# Create mappings for win percentages
home_team_win_percentage_map = home_team_win_percentage.to_dict()
away_team_win_percentage_map = away_team_win_percentage.to_dict()
# 
# Map the win percentages to the original DataFrame
df_cleaned['HomeTeamWinPercentage'] = df_cleaned['HomeTeam'].map(home_team_win_percentage_map).fillna(0)
df_cleaned['AwayTeamWinPercentage'] = df_cleaned['AwayTeam'].map(away_team_win_percentage_map).fillna(0)
# 
# Check the first few rows to confirm the new features
print(df_cleaned[['HomeTeam', 'HomeTeamWinPercentage', 'AwayTeam', 'AwayTeamWinPercentage']].head())

      HomeTeam  HomeTeamWinPercentage       AwayTeam  AwayTeamWinPercentage
0      Burnley               0.111111       Man City               0.555556
1      Arsenal               0.777778  Nott'm Forest               0.111111
2  Bournemouth               0.250000       West Ham               0.444444
3     Brighton               0.444444          Luton               0.125000
4      Everton               0.333333         Fulham               0.111111


In [16]:
# Create a new dataframe to store the league standings
league_standings = pd.DataFrame()

# Loop through each league
for league in df_cleaned['Div'].unique():
    # Initialize a DataFrame for this league
    league_df = df_cleaned[df_cleaned['Div'] == league].copy()
    
    # Calculate points for home and away games
    home_points = league_df[league_df['FTR'] == 'H'].groupby('HomeTeam')['FTR'].count() * 3
    away_points = league_df[league_df['FTR'] == 'A'].groupby('AwayTeam')['FTR'].count() * 3
    draw_points_home = league_df[league_df['FTR'] == 'D'].groupby('HomeTeam')['FTR'].count()
    draw_points_away = league_df[league_df['FTR'] == 'D'].groupby('AwayTeam')['FTR'].count()
    
    # Combine points for each team
    total_points = (home_points.add(away_points, fill_value=0)
                    .add(draw_points_home, fill_value=0)
                    .add(draw_points_away, fill_value=0)
                    .reset_index(name='Points'))
    total_points['Team'] = total_points.apply(lambda x: x['HomeTeam'] if pd.notna(x['HomeTeam']) else x['AwayTeam'], axis=1)
    
    # Calculate goal differences and goals scored for home and away games
    home_goal_diff = league_df.groupby('HomeTeam').apply(lambda x: (x['FTHG'] - x['FTAG']).sum()).reset_index(name='HomeGoalDiff')
    away_goal_diff = league_df.groupby('AwayTeam').apply(lambda x: (x['FTAG'] - x['FTHG']).sum()).reset_index(name='AwayGoalDiff')
    home_goals_scored = league_df.groupby('HomeTeam')['FTHG'].sum().reset_index(name='HomeGoalsScored')
    away_goals_scored = league_df.groupby('AwayTeam')['FTAG'].sum().reset_index(name='AwayGoalsScored')

    # Merge the statistics into one DataFrame
    team_stats = total_points.merge(home_goal_diff, left_on='Team', right_on='HomeTeam', how='left')
    team_stats = team_stats.merge(away_goal_diff, left_on='Team', right_on='AwayTeam', how='left')
    team_stats = team_stats.merge(home_goals_scored, left_on='Team', right_on='HomeTeam', how='left')
    team_stats = team_stats.merge(away_goals_scored, left_on='Team', right_on='AwayTeam', how='left')
    
    # Calculate total goal difference and goals scored
    team_stats['GoalDiff'] = team_stats['HomeGoalDiff'].fillna(0) + team_stats['AwayGoalDiff'].fillna(0)
    team_stats['GoalsScored'] = team_stats['HomeGoalsScored'].fillna(0) + team_stats['AwayGoalsScored'].fillna(0)
    
    # Sort teams by points and then by goal difference and goals scored
    team_stats.sort_values(by=['Points', 'GoalDiff', 'GoalsScored'], ascending=[False, False, False], inplace=True)

    # Add a 'League' column to identify the league of each team
    team_stats['League'] = league
    
    # Append to the main league_standings DataFrame
    league_standings = pd.concat([league_standings, team_stats[['Team', 'Points', 'GoalDiff', 'GoalsScored', 'League']]], ignore_index=True)

# Reset index if needed (optional)
league_standings.reset_index(drop=True, inplace=True)

# Now league_standings should have the cumulative points and standings for each team in their respective leagues
print(league_standings)

# To find Chelsea's position in the EPL
chelsea_stats = league_standings[(league_standings['Team'] == 'Chelsea') & (league_standings['League'] == 'E0')]
chelsea_position = league_standings[(league_standings['League'] == 'E0')].reset_index().index['Chelsea'].get_loc(chelsea_stats.index[0]) + 1

print(f"Chelsea is currently in position: {chelsea_position} in the EPL")

#Display Chelsea's stats
print(chelsea_stats)

KeyError: 'HomeTeam'